In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re


In [ ]:

driver = webdriver.Chrome()

try:
    # Abrir la página web
    url = "https://procesosjudiciales.funcionjudicial.gob.ec/busqueda-filtros"
    driver.get(url)

    # Esperar hasta que el campo de Actor/Ofendido esté presente en la página
    actor_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[formcontrolname="cedulaActor"]'))
    )

    # Enviar la información al campo de Actor/Ofendido
    actor_input.send_keys("0968599020001")  # Aquí se puede proporcionar la información que deseas buscar

    # Enviar la tecla 'Enter' para realizar la búsqueda
    actor_input.send_keys(Keys.RETURN)

    # Esperar hasta que se carguen los resultados de la búsqueda
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.accion-infraccion'))
    )

    # Abrir un archivo CSV para escribir los resultados
    with open('listado_procesos.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Resultado'])  # Escribir encabezado en el archivo CSV

        # Mientras haya resultados en la página actual
        current_page = 1
        while True:

            # Verificar si la página actual no es la primera página
            
                
            # Lista para almacenar la información recopilada de cada ícono
            textos_modificados = []
            iconos_folder_1 = driver.find_elements(By.XPATH, '//mat-icon[contains(text(), "folder_open")]')

            # Iterar sobre cada ícono de folder
            for i in range(len(iconos_folder_1)):
                if current_page != 1 and i != 0:
                    # Iterar desde la página actual hasta la última página
                    for page_number in range(current_page - 1):  # Reemplaza last_page con el número total de páginas
                        # Encontrar y hacer clic en el botón de "Página siguiente"
                        next_button = driver.find_element(By.XPATH, '//button[@aria-label="Página siguiente"]')
                        if next_button.is_enabled():
                            next_button.click()
                            time.sleep(2)  # Esperar a que se cargue la página
                        else:
                            break  # Salir del bucle si no hay más páginas disponibles

                #print(i)
                iconos_folder = driver.find_elements(By.XPATH, '//mat-icon[contains(text(), "folder_open")]')
                icono_folder = iconos_folder[i]
                icono_folder.click()

                # Esperar a que aparezca la información general
                results_generales = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ng-star-inserted'))
                )
                time.sleep(2)
                iconos_actuaciones = driver.find_elements(By.XPATH, '//mat-icon[contains(text(), "folder_open")]')
                actuaciones = []
                for icono_act in iconos_actuaciones:
                    icono_act.click()
                    results_act = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.mat-mdc-tooltip-trigger.title'))
                )
                
                    # Iterar sobre cada elemento encontrado
                    acts = []
                    for result_act_1 in results_act:
                        # Obtener el texto del elemento
                        texto = result_act1.get_attribute("textContent")
                        # Imprimir el texto
                        print(texto)
                        acts.append(texto)
                    actuaciones.append(acts)
                    icono_regresar = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//span[@class="mdc-button__label" and contains(text(), "Regresar")]'))
                    )
                    icono_regresar.click()

                    

                # Recopilar la información deseada y agregarla a la lista
                informacion_recopilada = []
                for result in results_generales:
                    lineas = result.text.split('\n')
                    info_dict = {}
                    for i in range(0, len(lineas), 2):
                        if i + 1 < len(lineas):
                            clave = lineas[i].strip()
                            valor = lineas[i+1].strip()
                            info_dict[clave] = valor
                    informacion_recopilada.append(info_dict)
                    informacion_recopilada_sin_vacios = [diccionario for diccionario in informacion_recopilada if diccionario]# Regresar a la página anterior
                textos_modificados.append(informacion_recopilada_sin_vacios)
                icono_regresar = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//span[@class="mdc-button__label" and contains(text(), "Regresar")]'))
                )
                icono_regresar.click()
                driver.execute_script("window.scrollBy(0, -100);")
                time.sleep(2)  # Escribir los resultados en el archivo CSV
            results = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.accion-infraccion'))
            )
            for result in results:
                writer.writerow([result.text])

            # Buscar el botón "Página siguiente"
            next_button = driver.find_element(By.XPATH, '//button[@aria-label="Página siguiente"]')

            # Hacer clic en el botón "Página siguiente" si está disponible
            if next_button.is_enabled():
                next_button.click()
                time.sleep(2)
                # Esperar hasta que se carguen los resultados de la página siguiente
                WebDriverWait(driver, 10).until(
                    EC.staleness_of(results[0])  # Esperar a que los resultados anteriores desaparezcan
                )
                # Reiniciar el índice para iterar sobre los íconos de carpeta en la siguiente página
                current_page += 1
                print(current_page)
                i = 0
            else:
                # Si no hay más páginas disponibles, salir del bucle
                break

finally:
    pass
   